In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('../data/train.csv')

In [3]:
train.head()

,user_id,movie_id,rating,timestamp,movie_title,genres,avg_rating
0,259,255,4,874724710,My Best Friend's Wedding (1997),Romance,4.0
1,259,286,4,874724727,"English Patient, The (1996)","Romance, War",4.0
2,259,298,4,874724754,Face/Off (1997),"Action, Sci-Fi, Thriller",4.0
3,259,185,4,874724781,Psycho (1960),"Horror, Romance, Thriller",4.0
4,259,173,4,874724843,"Princess Bride, The (1987)","Action, Adventure, Romance",4.0


In [25]:
train.groupby('user_id')['movie_title'].apply(list).apply(lambda x: len(x)).describe()

count    621.000000
mean     102.394525
std       94.180971
min        1.000000
25%       35.000000
50%       65.000000
75%      146.000000
max      737.000000
Name: movie_title, dtype: float64

In [26]:
train

,user_id,movie_id,rating,timestamp,movie_title,genres,avg_rating
0,259,255,4,874724710,My Best Friend's Wedding (1997),Romance,4.0
1,259,286,4,874724727,"English Patient, The (1996)","Romance, War",4.0
2,259,298,4,874724754,Face/Off (1997),"Action, Sci-Fi, Thriller",4.0
3,259,185,4,874724781,Psycho (1960),"Horror, Romance, Thriller",4.0
4,259,173,4,874724843,"Princess Bride, The (1987)","Action, Adventure, Romance",4.0
...,...,...,...,...,...,...,...
63582,642,8,5,885603662,Babe (1995),Children,5.0
63583,642,234,1,885603662,Jaws (1975),"Action, Horror",1.0
63584,642,969,2,885603662,Winnie the Pooh and the Blustery Day (1968),"Animation, Children",2.0
63585,642,181,5,885603699,Return of the Jedi (1983),"Action, Adventure, Romance, Sci-Fi, War",5.0


In [247]:
users_top_movies = train.groupby('user_id').apply(lambda x: x.sort_values('rating', ascending=False).head(10)['movie_title'].to_list())

In [61]:
user_ratings = train.drop_duplicates(['user_id', 'movie_title']).pivot(index='user_id', columns='movie_title', values='rating').fillna(0)

In [173]:
ten_closest_users = user_ratings.apply(
    lambda x: user_ratings.loc[user_ratings.index != x.name]
    .apply(lambda y: ((y - x) ** 2).sum(), axis=1)
    .sort_values()
    .head(10)
    .index.to_list(),
    axis=1,
)

In [174]:
ten_closest_users.head()

user_id
1      [933, 521, 715, 738, 868, 44, 538, 77, 198, 508]
5     [638, 275, 723, 700, 746, 55, 517, 231, 483, 563]
6     [321, 452, 123, 929, 553, 237, 338, 82, 492, 745]
8       [55, 153, 352, 37, 700, 517, 785, 80, 638, 746]
10      [321, 6, 123, 237, 426, 710, 908, 397, 398, 82]
dtype: object

In [235]:
def createPrompt(each):
    return f"""I am user {each}.  The users who are most like me are {ten_closest_users[each]}.
The top movies for each of these users are:
{"; ".join(pd.Series(users_top_movies.loc[ten_closest_users[each]].index).apply(lambda x: ": ".join([str(x), ", ".join(users_top_movies.loc[x])])).to_list())}."""

In [237]:
createPrompt(ten_closest_users.iloc[:1].index[0])

"I am user 1.  The users who are most like me are [933, 521, 715, 738, 868, 44, 538, 77, 198, 508].\nThe top movies for each of these users are:\n933: Silence of the Lambs, The (1991), Pulp Fiction (1994), Fargo (1996), Apocalypse Now (1979), Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963), Godfather, The (1972), Clockwork Orange, A (1971), Braveheart (1995), Shawshank Redemption, The (1994), Sting, The (1973), Dead Poets Society (1989), Platoon (1986), Butch Cassidy and the Sundance Kid (1969), Twelve Monkeys (1995), Die Hard (1988), Clueless (1995), GoodFellas (1990), Star Trek: The Wrath of Khan (1982), Usual Suspects, The (1995), Schindler's List (1993); 521: Mystery Science Theater 3000: The Movie (1996), Clerks (1994), Sneakers (1992), Chasing Amy (1997), Akira (1988), Usual Suspects, The (1995), Amadeus (1984), Happy Gilmore (1996), Clockwork Orange, A (1971), Pulp Fiction (1994), Arsenic and Old Lace (1944), Princess Bride, The (1987), Austin Powers:

In [256]:
new_line = '\n'
for each in ten_closest_users.iloc[:1].index:
    Input = f"""I am user {each}.  The users who are most like me are {ten_closest_users[each]}.
The top movies for each of these users are:
{new_line.join(pd.Series(users_top_movies.loc[ten_closest_users[each]].index).apply(lambda x: ": ".join([str(x), ", ".join(users_top_movies.loc[x])])).to_list())}.
Please recommend ten movies for me to watch:"""
    

In [257]:
Input

"I am user 1.  The users who are most like me are [933, 521, 715, 738, 868, 44, 538, 77, 198, 508].\nThe top movies for each of these users are:\n933: Silence of the Lambs, The (1991), Pulp Fiction (1994), Fargo (1996), Apocalypse Now (1979), Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963), Godfather, The (1972), Clockwork Orange, A (1971), Braveheart (1995), Shawshank Redemption, The (1994), Sting, The (1973)\n521: Mystery Science Theater 3000: The Movie (1996), Clerks (1994), Sneakers (1992), Chasing Amy (1997), Akira (1988), Usual Suspects, The (1995), Amadeus (1984), Happy Gilmore (1996), Clockwork Orange, A (1971), Pulp Fiction (1994)\n715: Schindler's List (1993), Patton (1970), Shawshank Redemption, The (1994), Apollo 13 (1995), Die Hard (1988), Groundhog Day (1993), Pulp Fiction (1994), North by Northwest (1959), Aliens (1986), Silence of the Lambs, The (1991)\n738: Star Trek: The Wrath of Khan (1982), Raiders of the Lost Ark (1981), Princess Bride, 

In [258]:
from openai import OpenAI
from key import OPENAI_API_KEY
client = OpenAI(api_key=OPENAI_API_KEY)

In [259]:
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages= [{ 'role':'user','content' : Input}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        )

In [265]:
response.choices[0].message.content

"Based on the movie preferences of users who are most like you, I recommend the following ten movies for you to watch:\n\n1. The Shawshank Redemption (1994)\n2. Pulp Fiction (1994)\n3. A Clockwork Orange (1971)\n4. Silence of the Lambs, The (1991)\n5. Raiders of the Lost Ark (1981)\n6. Apocalypse Now (1979)\n7. Star Wars (1977)\n8. Blade Runner (1982)\n9. Schindler's List (1993)\n10. Princess Bride, The (1987)\n\nEnjoy your movie marathon!"